<a href="https://colab.research.google.com/github/Ngueyap/Projet-HDH/blob/main/Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importer les bibliothèques nécessaires
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

# Créer une instance du moteur SQL
engine = create_engine('sqlite:///sante_data.db')

# Créer une session SQLAlchemy
Session = sessionmaker(bind=engine)
session = Session()

# Créer une classe de base pour les objets de la base de données
Base = declarative_base()


# Définir la classe pour la table "person"
class Person(Base):
    __tablename__ = 'person'

    person_id = Column(Integer, primary_key=True)
    person_source_value = Column(String)
    gender_concept_id = Column(Integer)
    year_of_birth = Column(Integer)  # Cette colonne sera remplie ultérieurement

# Créer la table dans la base de données
Base.metadata.create_all(engine)




<ipython-input-1-2bb82c6c71e2>:14: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [21]:
pd.read_csv('T_MCOaaB.csv')


,ETA_NUM,RSA_NUM,AGE_ANN,AGE_GES,AGE_JOU,ANT_SUP_NBR,AUT_PGV_NBR,BDI_COD,BDI_DEP,BEB_SEJ,...,FAIS_NBR,GHS_9611_ACT,GHS_9612_ACT,GHS_9510_ACT,GHS_9524_ACT,DLY_ACT,GHS_6523_ACT,NBR_SUP_SRA,COD_IGS,ANN_IVG_PREC
0,250010162,6766543446,99,12,,727,2,xXBaw,Rl,N,...,94642244,8915155,6792344,76916336,17976268,ziIi,16284601,69219080,EVe,1986
1,470015009,1778813122,11,21,,145,1,cEYAy,OB,R,...,25710439,88072131,55074162,36654384,49863395,gSqO,14751924,20508460,pPd,2008
2,930021001,7040927016,117,15,,659,2,qLjMe,rJ,D,...,63734687,64466581,90909296,21059978,16648114,Ibdk,66461546,71462718,eED,1979
3,130041403,4301299151,109,29,,398,1,EqJNQ,lN,r,...,19090100,27414396,85524004,46650539,29912523,SOnE,14877180,11779647,mjj,2002
4,770816072,7664212316,26,37,,684,1,iqpPr,AK,h,...,58862696,63682818,86159980,28424681,25949177,Ajfa,95191284,27466167,vxJ,1986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515,60017977,3508545843,80,30,,23,2,XDMBY,bp,A,...,55981785,65023461,2185605,80031727,250452,PtfZ,87742952,13950930,YxZ,1993
516,130039738,3247508060,86,38,,979,1,jHPYi,En,k,...,22873815,67865283,3626762,94841723,71977115,QMfi,13888143,86180847,aYM,2014
517,60019288,8729806187,112,32,,811,2,tuvEH,Jk,L,...,12249944,51531937,45157167,32496640,90317037,IneJ,76040961,88604805,PJV,2007
518,470012279,8052047654,60,40,,367,0,caXYm,nx,e,...,56108002,78548699,69723411,71868954,92116194,TBWt,2389081,3545902,djL,2010


In [23]:
import pandas as pd

class DataCleaner:
    def __init__(self, file_path):
        self.data = pd.read_csv(file_path)

    def get_num_rows_before_cleaning(self):
        return len(self.data)

    def remove_duplicates(self):
        self.data = self.data.drop_duplicates()

    def remove_missing_values(self):
        self.data = self.data.dropna()

    def get_cleaned_data(self):
        return self.data

    def get_distri_sexe(self):
        return self.data['COD_SEX'].value_counts()

    def map_sex_codes(self):
        # Remplacez les valeurs de COD_SEX selon votre codification
        sex_mapping = {1: 'Homme', 2: 'Femme', 0: 'Indéterminé', 9: 'Indéterminé'}
        self.data['COD_SEX'] = self.data['COD_SEX'].map(sex_mapping)



# Utilisation de la classe pour nettoyer deux fichiers CSV
if __name__ == '__main__':
    file_path1 = 'T_MCOaaC.csv'
    file_path2 = 'T_MCOaaB.csv'


    cleaner1 = DataCleaner(file_path1)
    num_rows_before_cleaning1 = cleaner1.get_num_rows_before_cleaning()
    cleaner1.remove_duplicates()
    cleaner1.remove_missing_values()
    cleaned_data1 = cleaner1.get_cleaned_data()



    cleaner2 = DataCleaner(file_path2)
    num_rows_before_cleaning2 = cleaner2.get_num_rows_before_cleaning()
    cleaner2.remove_duplicates()
    cleaner2.remove_missing_values()
    cleaned_data2 = cleaner2.get_cleaned_data()

    # Traitement de la colonne COD_SEX
    cleaner2.map_sex_codes()


    # nombres de lignes
    print(f"Nombre de lignes avant le nettoyage pour le fichier 1 : {num_rows_before_cleaning1}, "
          f"Nombre de lignes après le nettoyage : {len(cleaned_data1)}")
    print(f"Nombre de lignes avant le nettoyage pour le fichier 2 : {num_rows_before_cleaning2}, "
          f"Nombre de lignes après le nettoyage : {len(cleaned_data2)}")
    print("Distribution homme/femme dans T_MCOAAB :\n", cleaner2.get_distri_sexe())

Nombre de lignes avant le nettoyage pour le fichier 1 : 542, Nombre de lignes après le nettoyage : 514
Nombre de lignes avant le nettoyage pour le fichier 2 : 520, Nombre de lignes après le nettoyage : 513
Distribution homme/femme dans T_MCOAAB :
 Femme          190
Homme          165
Indéterminé    158
Name: COD_SEX, dtype: int64
